In [6]:
import bs4 as bs
import pickle
import requests

import datetime as dt
import os

import matplotlib.pyplot as plt
from matplotlib import style

import numpy as np

import pandas as pd
import pandas_datareader.data as  web

In [4]:
style.use('ggplot')
def visualize_data():
    df=pd.read_csv('sp500_joined_closes.csv')
    
    # cont corrlextion between database of file.
    df_corr=df.corr()   
    print(df_corr.head())
    
    #Drow plot from df_corr 
    
    data=df.corr.values
    fig=plt.figure()
    ax= fig.add_subplot(111)
    
    heatmap= ax.pcolor(data, cmap=plt.cm.RDLGn )
    fig.colorbar(heatmap)
    ax.set_xticks(np.arange(data.shape[0])+0.5, minor=False)
    ax.set_yticks(np.arange(data.shape[1])+0.5, minor=False)
    
    ax.invert_yaxis( )
    ax.xaxs.tick_top()
    
    column_labels=df_corr.columns
    row_lables=df.corr.ndex
    
    ax.set_xticklabels(column_labels)
    ax.set_yticklabels(row_labels)
    
    plt.xtickes(rotation=90)
    
    heatmap.set_clime(-1,1)
    plt.tight_layout()
    plt.show()

In [5]:
if __name__=="__main__":
    visualize_data()

FileNotFoundError: File b'sp500_joined_closes.csv' does not exist

In [7]:
import urllib2
import pytz
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime
from pandas.io.data import DataReader


SITE = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
START = datetime(1900, 1, 1, 0, 0, 0, 0, pytz.utc)
END = datetime.today().utcnow()


def scrape_list(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(site, headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)

    table = soup.find('table', {'class': 'wikitable sortable'})
    sector_tickers = dict()
    for row in table.findAll('tr'):
        col = row.findAll('td')
        if len(col) > 0:
            sector = str(col[3].string.strip()).lower().replace(' ', '_')
            ticker = str(col[0].string.strip())
            if sector not in sector_tickers:
                sector_tickers[sector] = list()
            sector_tickers[sector].append(ticker)
    return sector_tickers


def download_ohlc(sector_tickers, start, end):
    sector_ohlc = {}
    for sector, tickers in sector_tickers.iteritems():
        print 'Downloading data from Yahoo for %s sector' % sector
        data = DataReader(tickers, 'yahoo', start, end)
        for item in ['Open', 'High', 'Low']:
            data[item] = data[item] * data['Adj Close'] / data['Close']
        data.rename(items={'Open': 'open', 'High': 'high', 'Low': 'low',
                           'Adj Close': 'close', 'Volume': 'volume'},
                    inplace=True)
        data.drop(['Close'], inplace=True)
        sector_ohlc[sector] = data
    print 'Finished downloading data'
    return sector_ohlc


def store_HDF5(sector_ohlc, path):
    with pd.get_store(path) as store:
        for sector, ohlc in sector_ohlc.iteritems():
            store[sector] = ohlc


def get_snp500():
    sector_tickers = scrape_list(SITE)
    sector_ohlc = download_ohlc(sector_tickers, START, END)
    store_HDF5(sector_ohlc, 'snp500.h5')


if __name__ == '__main__':
    get_snp500()


SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Downloading data from Yahoo for %s sector' % sector)? (<ipython-input-7-fbecfa4d111e>, line 37)

In [13]:
import googlefinance.client
help(googlefinance.client)

Help on module googlefinance.client in googlefinance:

NAME
    googlefinance.client - # coding: utf-8

FUNCTIONS
    get_closing_data(queries, period)
    
    get_open_close_data(queries, period)
    
    get_price_data(query)
    
    get_prices_data(queries, period)
    
    get_prices_time_data(queries, period, interval)

FILE
    /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/googlefinance/client.py




In [ ]:
def GET_DATA_FROM_GOOGLE(reload_sp500=False ):
    if reload_sp500:
         tickers=SAVE_SP500_TICKERS()
    else:
        with open('sp500tickers.pickle','rb') as f:
            tickers=pickle.load(f)
    if not os.path.exists('stock_dfs '):
        os.makedirs('stock_dfs') 
        
        
    start=dt.datetime(2012,1,1)
    #end=dt.datetime(2017,1,1)
    end   = dt.date.today()
    
    for ticker in tickers:
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker) ):
            df=web.DataReader(ticker,'morningstar',start,end)
            #param = {
                #'q': "/{}".format(ticker) # Stock symbol (ex: "AAPL")
                #'i': "86400", # Interval size in seconds ("86400" = 1 day intervals)
                #'x': "NASD", # Stock exchange symbol on which stock is traded (ex: "NASD")
                #'p': "1Y" # Period (Ex: "1Y" = 1 year)
            #}
            #df = get_price_data(param)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else :
            print('ALready have {}'.format(tikcer ))
        

In [15]:
import bs4 as bs
import pickle
import requests

import datetime as dt
import os
import numpy as np

from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data
import pandas as pd
import pandas_datareader.data as  web

import time

In [1]:
import quandl

In [4]:
def SAVE_SP500_TICKERS():
    resp=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup=bs.BeautifulSoup(resp.text,'lxml')
    table=soup.find('table',{'class':"wikitable sortable"})
    tickers=[]
    for row in table.findAll('tr')[1:]:
        ticker=row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open('sp500tickers.pickle','wb') as f:
        pickle.dump(tickers,f)
        
    #print(tickers)
    
    return tickers


sa=SAVE_SP500_TICKERS()
print( len(sa))

505


In [ ]:
import bs4 as bs
import pickle
import requests

import bs4 as bs
import datetime as dt
import os
import pandas_datareader.data as web
import pickle
import requests

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)

    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        

# save_sp500_tickers()
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'morningstar', start, end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            df = df.drop("Symbol", axis=1)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))


get_data_from_yahoo()

adding ANDV to retry list


In [28]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests
import quandl

import time
import fix_yahoo_finance as yf  
from iexfinance import get_historical_data

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers


# save_sp500_tickers()
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2015, 1, 1)
    end = dt.datetime.now()
    for ticker in range(200,len(tickers)):
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(tickers[ticker])):
            #df = web.DataReader(tickers[ticker], 'morningstar', start, end)
            
            #df = quandl.get("EOD/{}".format(tickers[ticker]), authtoken="KxKziu7vxQ91gSPCxGnj")
            #df = yf.download(tickers[ticker],start,end)
            #df = quandl.get("WIKI/{}".format(tickers[ticker]),start_date=start,end_date=end)
            
            df = get_historical_data(tickers[ticker], start=start, end=end, output_format='pandas')
            df.reset_index(inplace=True)
            #df.set_index("Date", inplace=True)
            #df = df.drop("Symbol", axis=1)
            df.to_csv('stock_dfs/{}.csv'.format(tickers[ticker]))
            time.sleep(30)
            print(ticker)
        else:
            print('Already have {}'.format(tickers[ticker]))


            
get_data_from_yahoo()

Already have FLIR
Already have FLS
Already have FLR
Already have FMC
Already have FL
Already have F
Already have FTV
Already have FBHS
Already have BEN
Already have FCX
Already have GPS
Already have GRMN
Already have IT
Already have GD
Already have GE
Already have GGP
Already have GIS
Already have GM
Already have GPC
Already have GILD
Already have GPN
Already have GS
Already have GT
Already have GWW
Already have HAL
Already have HBI
Already have HOG
Already have HRS
Already have HIG
Already have HAS
Already have HCA
Already have HCP
Already have HP
Already have HSIC
Already have HSY
Already have HES
Already have HPE
Already have HLT
Already have HFC
Already have HOLX
Already have HD
Already have HON
Already have HRL
Already have HST
Already have HPQ
Already have HUM
Already have HBAN
Already have HII
Already have IDXX
Already have INFO
Already have ITW
Already have ILMN
Already have IR
Already have INTC
Already have ICE
Already have IBM
Already have INCY
Already have IP
Already have IP

In [ ]:
## https://www.quora.com/Using-Python-whats-the-best-way-to-get-stock-data